In [16]:
!python3 --version

Python 3.8.10


In [ ]:
from pyspark.sql import SparkSession

In [73]:
import time
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

def subgraph2vec(spark_session, data_file_path=None):
    schema = StructType([
        StructField("source", IntegerType(), True),
        StructField("target", IntegerType(), True),
        StructField("edge_type", StringType(), True)
    ])

    data_file_path = "/opt/spark/data/large_graph.txt" # Giữ nguyên path cố định hoặc dùng tham số
    
    df = spark_session.read.option("delimiter", "|").option("header", "false") \
        .schema(schema).csv(data_file_path)
    
    total_records = df.count()
    print(f"📋 Total edges processed: {total_records:,}")

    df_repartitioned = df.repartition(10)
    def process_partition_and_write_out(partition_iterator):
        for record in partition_iterator:
            time.sleep(0.001)  # Giả lập xử lý
            print('Hihi')
        return []

    df_processed = df_repartitioned.rdd.mapPartitions(process_partition_and_write_out)

    return df_processed.toDF(schema)

In [ ]:

spark = None
try:
    spark = SparkSession.builder \
        .appName("SimpleGraphProcessing_v3") \
        .master("spark://spark-master:7077") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()
    graph_df = subgraph2vec(spark)
    graph_df.show()
    print("✅ Graph processing completed successfully!")
except Exception as e:
    import traceback
    traceback.print_exc()    
finally:
    if spark:
        spark.stop()

📋 Total edges processed: 100,001
